ref : https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/?utm_source=chatgpt.com

#Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/swuds/t5_thesis/98_progress/dataset/nd_sales_Prophet.csv')
print(df.head())

#Preprocess

In [ ]:
# กำหนดลิสต์ของ Regressor ที่จะใช้
regressor_columns = ['ismarketday', 'isschoolday', 'holiday', 'isbuddaday', 'nd_lottery']

# เลือกคอลัมน์ที่จำเป็นทั้งหมด
df_prophet = df[['ds', 'y'] + regressor_columns].copy()
print("Data prepared for Prophet with regressors.")

In [ ]:
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

In [ ]:
train_prophet = df_prophet.iloc[:-7]
test_prophet = df_prophet.iloc[-7:]

#Train

In [ ]:
model = Prophet(seasonality_mode='additive')

# --- ส่วนที่แก้ไข 2: เพิ่ม Regressor ทุกตัวเข้าไปในโมเดล ---
for regressor in regressor_columns:
    model.add_regressor(regressor)

# เทรนโมเดลด้วยข้อมูลทั้งหมด (y และ regressors)
model.fit(train_prophet)
print("\nProphet model with regressors trained successfully.")

In [ ]:
# 1. สร้าง DataFrame สำหรับอนาคตเผื่อไว้ (เช่น 9 วัน) เพื่อให้แน่ใจว่าหลังกรองแล้วจะเหลืออย่างน้อย 7 วัน
# (9 วัน จะครอบคลุมวันเสาร์ได้อย่างน้อย 1 วันเสมอ)
future = model.make_future_dataframe(periods=9, freq='D')

# เพราะ df_prophet คือตัวที่เราเตรียมไว้สำหรับ Prophet และมีข้อมูลที่ถูกต้องครบถ้วน
future_with_regressors = pd.merge(future, df_prophet[['ds'] + regressor_columns], on='ds', how='left')

# 3. เติมค่าว่าง (NaN) สำหรับ Regressor ในอนาคต
future_with_regressors.fillna(method='ffill', inplace=True)
#print("\nFuture dataframe created and future regressors have been filled.")

# 2. กรองวันเสาร์ออก
future_no_saturday = future_with_regressors[future_with_regressors['ds'].dt.dayofweek != 5]

# 3. **เลือกมาเฉพาะ 7 วันแรกจริงๆ ที่ไม่ใช่วันเสาร์ เพื่อทำนาย**
# โดยเราจะเลือกข้อมูลทั้งหมดในอดีต บวกกับ 7 วันแรกในอนาคตที่กรองแล้ว
future_to_predict = future_no_saturday.head(len(train_prophet) + 7)
print("Dates to be predicted (7 days, excluding Saturdays):")
print(future_to_predict.tail(7))

In [ ]:
# 5. ทำนายโดยใช้ DataFrame ที่สมบูรณ์แล้ว
forecast = model.predict(future_no_saturday)

#Evaluate

In [ ]:
# prompt: ประเมินผลด้วย rmse, mae

# Filter forecast and test data to include only the 7 days being evaluated
# Ensure that the dates in `test_prophet` match the dates in the last 7 predictions
test_dates = test_prophet['ds'].values

# Find the corresponding predictions in the forecast DataFrame
forecast_eval = forecast[forecast['ds'].isin(test_dates)]

# Make sure the test data and forecast data are aligned by date before calculating metrics
# Sort both DataFrames by date
test_prophet_sorted = test_prophet.sort_values(by='ds')
forecast_eval_sorted = forecast_eval.sort_values(by='ds')

# Ensure the lengths match and dates are identical
if not np.array_equal(test_prophet_sorted['ds'].values, forecast_eval_sorted['ds'].values):
    print("Warning: Dates in test data and forecast data do not match for evaluation.")
    print("Test Dates:\n", test_prophet_sorted['ds'].values)
    print("Forecast Dates:\n", forecast_eval_sorted['ds'].values)
    # You might want to handle this error appropriately, e.g., by raising an exception
    # For now, we'll proceed with the available matching dates, but be aware of potential issues
    common_dates = np.intersect1d(test_prophet_sorted['ds'].values, forecast_eval_sorted['ds'].values)
    test_prophet_sorted = test_prophet_sorted[test_prophet_sorted['ds'].isin(common_dates)]
    forecast_eval_sorted = forecast_eval_sorted[forecast_eval_sorted['ds'].isin(common_dates)]


y_true = test_prophet_sorted['y'].values
y_pred = forecast_eval_sorted['yhat'].values

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f'RMSE: {rmse:.3f}')

# Calculate MAE
mae = mean_absolute_error(y_true, y_pred)
print(f'MAE: {mae:.3f}')

In [ ]:
# prompt: plot line graph 7 วันสุดท้ายที่ forecast แกน x  = date, แกน y = sales_sum, Actual sales = เส้นทึบสีฟ้ามีจุดกลม, Prophet forecast = เส้นทึบสีแดงมีกากะบาท

# Filter the forecast to the last 7 days corresponding to the test set
forecast_last_7_days = forecast[forecast['ds'].isin(test_prophet['ds'])]

# Plotting
plt.figure(figsize=(10, 6))

# Plot Actual Sales (last 7 days)
plt.plot(test_prophet['ds'], test_prophet['y'], marker='o', linestyle='-', color='blue', label='Actual Sales')

# Plot Prophet Forecast (last 7 days)
plt.plot(forecast_last_7_days['ds'], forecast_last_7_days['yhat'], marker='x', linestyle='-', color='red', label='Prophet Forecast')

# Set labels and title
plt.xlabel('Date')
plt.ylabel('Sales Sum')
plt.title('Actual Sales vs. Prophet Forecast (Last 7 Days)')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# prompt: plot line graph 7 วันสุดท้ายที่ forecast แกน x  = date, แกน y = sales_sum, Actual sales = เส้นทึบสีฟ้ามีจุดกลม, Prophet forecast = เส้นทึบสีแดงมีกากะบาท

# กรองเฉพาะข้อมูล 7 วันสุดท้ายจากทั้ง Actual และ Forecast
last_7_days_actual = actuals_filtered.tail(7)
last_7_days_forecast = predictions_filtered.tail(7)

# พล็อต
plt.figure(figsize=(12, 6))

# พล็อต Actual Sales (เส้นทึบสีฟ้ามีจุดกลม)
plt.plot(last_7_days_actual['ds'], last_7_days_actual['y'], label='Actual sales', marker='o', linestyle='-', color='blue')

# พล็อต Prophet Forecast (เส้นทึบสีแดงมีกากะบาท)
plt.plot(last_7_days_forecast['ds'], last_7_days_forecast['yhat'], label='Prophet forecast', marker='X', linestyle='-', color='red')

# ตั้งชื่อแกนและหัวข้อ
plt.xlabel('Date')
plt.ylabel('sales_sum')
plt.title('Actual vs Prophet Forecast for Last 7 Days')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45) # หมุนชื่อแกน x เพื่อให้อ่านง่าย
plt.tight_layout() # ปรับระยะห่างระหว่างองค์ประกอบในกราฟ
plt.show()

In [ ]:
forecast_last_7_days